In [1]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import folium
import requests
from tqdm import tqdm
from collections import deque
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('List_of_postal_codes_of_Canada:_M.html', 'w', encoding="utf-8") as fo:
    fo.write(article)

Use BeautifulSoup to find all the tables

In [3]:
from bs4 import BeautifulSoup

# Load article, turn into soup and get the <table>s.
article = open('List_of_postal_codes_of_Canada:_M.html', encoding="utf8").read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

Search through the tables for the one with the headings we want

In [4]:
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']:
        break

Build the dataframe row by row, then manipulate the data into the shape we want

In [5]:
import pandas as pd

table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

df = pd.DataFrame(res, columns=['Postcode', 'Borough', 'Neighbourhood'])

# Drop the rows with Borough 'Not assigned'
df_filt = df[df['Borough'] != 'Not assigned']

# Concatenate the neighbourhoods of boroughs with multiple entries
df_filt_g = df_filt.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

# Replace 'Not assigned' neighbourhoods with the Borough
df_filt_g['Neighbourhood'].replace('Not assigned', df_filt_g['Borough']).reset_index()
df_filt_g

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
df_filt_g.shape

(103, 3)

In [7]:
# Read in CSV of locations of postal codes

locations = pd.read_csv('https://cocl.us/Geospatial_data')

# Rename 'Postal Code' to 'Postcode'
locations.columns = ['Postcode', 'Latitude', 'Longitude']

In [8]:
locations.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df_filt_g_locs = pd.merge(df_filt_g, locations, on = 'Postcode')
df_filt_g_locs.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Create a map of Toronto with neighbourhoods superimposed on top

In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_filt_g_locs['Latitude'], df_filt_g_locs['Longitude'], df_filt_g_locs['Borough'], df_filt_g_locs['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto